In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.common.exceptions import StaleElementReferenceException, ElementNotInteractableException,NoSuchElementException
from browsermobproxy import Server
import re
import time
import csv
import json
import hashlib

In [23]:

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach',True)
chrome_options.add_argument('--enable-logging')
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
#log_entries = driver.get_log("performance")
driver = webdriver.Chrome(options=chrome_options)
url = "https://underwood1.yonsei.ac.kr/com/lgin/SsoCtr/initExtPageWork.do?link=handbList&locale=eng"
driver.get(url)
driver.execute_cdp_cmd('Network.enable', {})


{}

In [24]:
# 대학(원) 분류 및 개설 학과 등등 조건 설정
labels = driver.find_element(By.CLASS_NAME, "cl-layout-content")

# 대학(원) 분류 선택 태그 id: uuid-5i
classification_combobox = labels.find_element(By.ID, "uuid-5i")
cc_combobox_button = classification_combobox.find_element(By.CLASS_NAME, "cl-combobox-button")
opened_course_combobox = labels.find_element(By.ID, "uuid-5k")
opened_course_button = opened_course_combobox.find_element(By.CLASS_NAME, "cl-combobox-button")

# driver.execute_script(f'arguments[0].setAttribute("{attribute_name}", "{attribute_value}")', element)


# 개설학과 분류 선택 태그 id: uuid-5k


In [25]:

def get_courses():
    parent_container = driver.find_element(By.CLASS_NAME, "cl-grid-centersplit")
    child_elements = parent_container.find_elements(By.XPATH,"./child::div")[1]
    child_elements = child_elements.find_element(By.XPATH,"./child::div")
    child_elements = child_elements.find_element(By.XPATH,"./child::div")
    child_elements = child_elements.find_element(By.XPATH,"./child::div")
    child_elements = child_elements.find_element(By.XPATH,"./child::div")
    scroll_origin = ScrollOrigin.from_element(child_elements)
    courses = child_elements.find_elements(By.XPATH,'.//div[@role="row"]')
    return [courses, scroll_origin]
    

In [26]:
xpath_expression = f"//*[contains(text(), '{"조회"}')]"
search_button = labels.find_elements(By.XPATH, xpath_expression)[0]


# 8개씩 나오는데 내려가면서 다 봐야 함.     

In [27]:
# 총건수
def get_search_nums():
    search_div = driver.find_element(By.XPATH, '//div[@aria-label[contains(., "총건수")]]')
    search_nums = search_div.get_attribute("aria-label")
    search_nums =  re.search(r'\[([\d]+)\]', search_nums)
    search_nums = int(search_nums.group(1))
    return search_nums

In [28]:
def generate_course_id(professor_name, subject_code):
    # 문자열을 합쳐서 UTF-8로 인코딩
    input_string = professor_name + subject_code
    input_bytes = input_string.encode('utf-8')
    # hashlib을 사용하여 해시 생성
    hashed = hashlib.sha256(input_bytes).hexdigest()
    # 원하는 형태의 ID로 가공 (예: 처음 8자리만 사용)
    course_id = hashed[:10]
    return course_id

In [29]:
def course_desc(course):
    columns = course.find_elements(By.XPATH, "./*")
    # id
    course_code = columns[7].text
    instructor = columns[15].text
    instructor = '-' if (instructor == '') else instructor
    course_id = generate_course_id(instructor, course_code)
    time.sleep(3)
    columns[9].click()
    time.sleep(3)
    log_entries = driver.get_log("performance")
    response_result = ''
    for entry in log_entries:
        try:
            message_obj = json.loads(entry.get("message"))
            message = message_obj.get("message")
            method = message.get("method")
 
            if method == 'Network.responseReceived':
                response_url = message.get('params',{}).get('response',{}).get('url','')
                request_id = message.get('params', {}).get('requestId', '')
                if "findSubjtDescList" in response_url:
                    response = driver.execute_cdp_cmd('Network.getResponseBody',{'requestId':request_id})
                    response_body = response.get('body',{})
                    response_result = json.loads(response_body)["_METADATA_"]["result"]
                    break
                    
                    
                    

        except Exception as e:
            print(e)
    time.sleep(2)
    
    # Perform actions with the found element (e.g., click)
    try:
        close_button = driver.find_element(By.XPATH,"//*[text()='닫기']")
        driver.execute_script("arguments[0].click();", close_button)
    except  NoSuchElementException :
        pass
    finally:
        time.sleep(1.5)
        result = {
            "course_id" : course_id,
            "course_data" : response_result
        }
        
        return result

In [30]:
# 추출 템플릿 및 dictionary 함수화 시켜야 함. 
def extract_course(course):
    columns = course.find_elements(By.XPATH, "./*")
    semester = columns[1].text
    campus = columns[2].text
    major = columns[3].text
    grade = columns[4].text
    subject_type = columns[5].text
    session = columns[6].text
    course_code = columns[7].text
    credits = columns[12].text
    course_name = columns[13].text
    is_cancelled = columns[14].text
    instructor = columns[15].text
    schedule = columns[16].text
    classroom = columns[17].text

    # columns[9] => 강의 정보
    
    course_info = {
        'semester': semester,
        'campus': campus,
        'major': major,
        'grade': grade,
        'subject_type': subject_type,
        'session': session,
        'course_code': course_code,
        'credits': credits,
        'course_name': course_name,
        'is_cancelled': is_cancelled,
        'instructor': instructor,
        'schedule': schedule,
        'classroom': classroom
    }


    return course_info


In [11]:
fieldnames = ['semester',
                'campus',
                'major',
                'grade',
                'subject_type',
                'session',
                'course_code',
                'credits',
                'course_name',
                'is_cancelled',
                'instructor',
                'schedule',
                'classroom']

In [31]:
course_data_fieldnames = ['course_id', 'course_data']

In [36]:

prev_oc_id = None
oc_id = None
for i in range(35):
    cc_combobox_button.send_keys(Keys.DOWN)
    cc_combobox_button.click()
    time.sleep(1)
    # keys_down (클릭한 상태에서 눌렀을 때 )
    opened_course_button.click()
    while (True):
        opened_course_button.send_keys(Keys.DOWN)
        opened_course_button.click()
        oc_id = opened_course_combobox.find_element(By.TAG_NAME, "input").get_attribute("aria-activedescendant")
        if (prev_oc_id is not None and prev_oc_id == oc_id):
            prev_oc_id = None
            oc_id = None
            break
        # 검색하고 결과값 크롤링 => csv 파일에 넣기!
        

        # 검색
        search_button.click()
        time.sleep(3)
        courses = get_courses()
        search_nums = get_search_nums()
        done = True 
        current_idx = 0
        with open("course_data_please.csv", "a+", newline='') as csv_file:
            
            writer = csv.DictWriter(csv_file, fieldnames=course_data_fieldnames)
            writer.writeheader()
            while done:
                try:
                    courses, scroll_origin = get_courses()
                    if (current_idx == search_nums):
                        done = False
                    for course in courses:
                        course_idx = int(course.get_attribute("aria-rowindex"))
                        if (course_idx > current_idx):
                            try:
                                writer.writerow(course_desc(course))
                                current_idx += 1
                            except ElementNotInteractableException:
                                continue
                    ActionChains(driver)\
                    .scroll_from_origin(scroll_origin, 0, 350)\
                    .perform()
                    time.sleep(1)  # 스크롤이 완료될 때까지 잠시 대기
            
            
                except StaleElementReferenceException:
                    pass

        print("Section Completed ✅")


        prev_oc_id = oc_id
            


Section Completed ✅
Section Completed ✅


ElementClickInterceptedException: Message: element click intercepted: Element <div class="cl-grid-cell cl-grid-cell-inherit" role="gridcell" data-role="gridcell" aria-colindex="10" data-cellindex="24" tabindex="-1" style="box-sizing: border-box; display: inline-block; height: 30px; left: 445px; overflow: hidden; position: absolute; top: 30px; width: 35px;">...</div> is not clickable at point (499, 627). Other element would receive the click: <div class="cl-overlay" style="bottom:0;left:0;position:absolute;right:0;top:0;" data-ndid="9uk"></div>
  (Session info: chrome=121.0.6167.85)
Stacktrace:
0   chromedriver                        0x00000001053967dc chromedriver + 4040668
1   chromedriver                        0x000000010538e9e0 chromedriver + 4008416
2   chromedriver                        0x0000000105001870 chromedriver + 284784
3   chromedriver                        0x000000010504af60 chromedriver + 585568
4   chromedriver                        0x00000001050493f0 chromedriver +